In [1]:
from xai.dqn import DQN
         

dqn = DQN(translate=True, rotate=False)     

/home/eivind/master-project/.venv/lib/python3.11/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
dqn.train(
    alpha=.1,
    buffer_size=20000,
    num_episodes=100,
    epsilon=.4,
    gamma=.9,
    steps=5,
    batch_size=300
)

In [3]:
dqn.play(show=True)